<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7d35d5c4a5b530fae7b535d5aee93aa7c2b528e91700391c7228f49a035a2eca
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 15:44:29
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 27.12 K (0.2%)
Current PnL: -29.12 L (-18.81%)
CY Booked + Current PnL: -14.83 L (-9.58%)
-------------------
Total profit:  1.14 L
Total loss:  -30.26 L
-------------------
Total Booked + Current PnL: 12.15 L (9.5%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.76 L (67.38%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.47,-18.03,22.11,0.09,18265.0,-18168.0,82609.0,91.68,32.0,M-SC,2.19,253.0,-0.99,0.62,8.53,OX40N,NTT,DURABLES
50,MASFIN,398.61,1.91,-2.94,25.74,22.05,24479.0,-2880.0,95100.0,-16.44,54.0,H-SC,7.04,164.0,-0.12,0.71,38.23,XR,ATH,FINANCE
43,ITC,452.00,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-0.24,-14.69,17.25,0.02,25687.0,-25647.0,148912.0,127.12,46.0,M-SC,16.63,234.0,-1.00,1.11,29.53,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.81,3.78,15.81,20.19,28602.0,6593.0,180913.0,-18.61,40.0,X-MC,5.97,68.0,0.23,1.35,19.47,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELAXO,1176.00,5.79,-45.49,179.40,52.31,141957.0,-66031.0,79129.0,-41.92,53.0,X-SC,5.90,91.0,-0.47,0.59,5.92,X40N,NTT,FOOTWEAR
61,ROUTE,2227.21,5.42,-47.51,222.04,69.05,155099.0,-63214.0,69852.0,-57.43,50.0,H-SC,23.10,140.0,-0.41,0.52,7.73,SR,ATH,IT
10,BANDHANBNK,400.00,4.08,-32.17,183.69,92.42,347064.0,-89619.0,188940.0,32.90,34.0,H-SC,7.85,170.0,-0.26,1.41,9.19,XY24,NTT,BANKS
56,RAJESHEXPO,518.00,4.03,-63.60,174.92,0.07,87339.0,-87246.0,49931.0,1729.55,53.0,L-SC,2.35,268.0,-1.00,0.37,24.18,OX40N,NTT,JEWELLERY
11,BATAINDIA,2096.00,3.76,-36.51,112.96,35.20,92276.0,-46981.0,81689.0,7.15,39.0,X-SC,15.17,92.0,-0.51,0.61,3.76,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,-4.52,-2.27,23.76,20.95,58461.0,-5720.0,246048.0,-3.30,49.0,X-LC,19.01,31.0,-0.10,1.83,32.55,X40,ATH,PAINTS
49,LAOPALA,464.00,-3.26,-40.09,132.17,39.09,103808.0,-52564.0,78541.0,72.04,23.0,H-SC,7.43,155.0,-0.51,0.59,1.15,AR,NTT,CERAMICS
62,SATIN,274.00,-2.35,-23.94,99.64,51.83,209374.0,-66154.0,210130.0,-31.67,26.0,H-SC,1.60,146.0,-0.32,1.57,2.36,XY24,NTT,FINANCE
2,ABBOTINDIA,35195.00,-2.26,-6.32,25.36,17.44,35599.0,-9472.0,140375.0,-19.42,32.0,X-MC,4.58,63.0,-0.27,1.05,11.62,X40,ATH,PHARMA
28,GREENPANEL,537.00,-1.92,-39.18,135.33,43.14,150699.0,-71721.0,111357.0,149.35,22.0,M-SC,8.65,240.0,-0.48,0.83,4.75,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.29,-7.99,125.97,107.91,171125.0,-11798.0,135846.0,-25.01,31.0,M-SC,11.87,216.0,-0.07,1.01,1.29,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.24,-14.69,17.25,0.02,25687.0,-25647.0,148912.0,127.12,46.0,M-SC,16.63,234.0,-1.00,1.11,29.53,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.47,-18.03,22.11,0.09,18265.0,-18168.0,82609.0,91.68,32.0,M-SC,2.19,253.0,-0.99,0.62,8.53,OX40N,NTT,DURABLES
66,SIS,528.00,0.69,-24.66,60.95,21.26,50987.0,-27379.0,83653.0,1973.06,46.0,H-SC,4.59,166.0,-0.54,0.62,13.02,OX40N,NTT,MISC
47,KANSAINER,340.00,0.03,-25.94,53.22,13.47,106290.0,-69948.0,199719.0,-69.48,26.0,H-SC,3.22,158.0,-0.66,1.49,1.55,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.52,5.58,63.16,72.27,112993.0,9452.0,178900.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.1,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.52,5.58,63.16,72.27,112993.0,9452.0,178900.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.10,XR,NTT,IT
38,INDIAMART,4810.62,-0.69,-2.98,113.07,106.72,135303.0,-3673.0,119663.0,-51.96,33.0,H-SC,1.19,139.0,-0.03,0.89,19.70,AR,ATH,MISC
25,FINCABLES,1641.55,1.29,-7.99,125.97,107.91,171125.0,-11798.0,135846.0,-25.01,31.0,M-SC,11.87,216.0,-0.07,1.01,1.29,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.47,-2.96,39.76,35.63,80862.0,-6199.0,203375.0,-16.74,35.0,H-MC,3.16,119.0,-0.08,1.52,13.22,AR,ATH,PHARMA
37,IEX,219.00,-0.51,-4.18,55.11,48.63,105588.0,-8359.0,191595.0,-36.24,48.0,H-SC,14.09,136.0,-0.08,1.43,6.89,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,2.01,-21.86,89.99,48.45,140986.0,-43837.0,156669.0,-27.39,22.0,X-MC,11.17,64.0,-0.31,1.17,2.01,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.34,-13.19,40.06,21.58,48734.0,-18486.0,121653.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.70,X40N,ATH,IT
75,TMPV,600.00,-1.03,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.32,-10.27,26.69,13.67,61560.0,-26413.0,230650.0,-16.94,29.0,X-LC,2.98,14.0,-0.43,1.72,8.89,XY25,NTT,FMCG
53,PAGEIND,51605.07,-0.11,-9.14,38.74,26.07,57637.0,-14960.0,148780.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.82,-22.13,97.11,53.50,228187.0,-66773.0,234978.0,-62.35,31.0,X-MC,1.24,58.0,-0.29,1.75,4.37,XY24,NTT,FMCG
43,ITC,452.0,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG
3,ACC,3906.0,-0.61,-24.98,118.82,64.15,212094.0,-59451.0,178500.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.13,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.52,-80.36,535.36,24.79,264827.0,-202389.0,49467.0,-81.32,40.0,X-SC,1.85,86.0,-0.76,0.37,20.93,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.19,3.38,26.68,30.97,48748.0,5977.0,182715.0,-20.92,55.0,X-MC,2.06,75.0,0.12,1.36,16.37,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.11,-9.14,38.74,26.07,57637.0,-14960.0,148780.0,-33.02,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-1.03,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-1.01,-5.90,42.15,33.76,79651.0,-11850.0,188970.0,-33.58,34.0,X-MC,5.09,57.0,-0.15,1.41,0.09,X40,ATH,FMCG
3,ACC,3906.00,-0.61,-24.98,118.82,64.15,212094.0,-59451.0,178500.0,-56.31,35.0,X-SC,1.79,82.0,-0.28,1.33,0.13,XY24,BTT,CEMENT
31,HAVELLS,2069.16,0.89,-9.10,45.74,32.48,136377.0,-29833.0,298158.0,-13.63,41.0,X-MC,5.18,67.0,-0.22,2.22,1.29,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.56,-29.96,103.95,42.84,47326.0,-19478.0,45528.0,-54.27,41.0,X-SC,37.02,83.0,-0.41,0.34,3.00,XY24,NTT,MISC
15,CAMS,4762.00,-0.52,-80.36,535.36,24.79,264827.0,-202389.0,49467.0,-81.32,40.0,X-SC,1.85,86.0,-0.76,0.37,20.93,X40N,NTT,MISC
59,RELAXO,1176.00,5.79,-45.49,179.40,52.31,141957.0,-66031.0,79129.0,-41.92,53.0,X-SC,5.90,91.0,-0.47,0.59,5.92,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,3.76,-36.51,112.96,35.20,92276.0,-46981.0,81689.0,7.15,39.0,X-SC,15.17,92.0,-0.51,0.61,3.76,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.34,-13.19,40.06,21.58,48734.0,-18486.0,121653.0,-56.26,24.0,X-MC,5.46,56.0,-0.38,0.91,7.70,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.87,-12.83,36.83,19.28,111072.0,-44376.0,301580.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.49,X40,ATH,IT
41,INFY,2275.00,-0.73,8.85,42.28,54.87,144000.0,27699.0,340587.0,-15.29,64.0,X-LC,3.38,2.0,0.19,2.54,17.87,X40,BTT,IT
75,TMPV,600.00,-1.03,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.37,-4.75,42.43,35.67,127650.0,-14993.0,300849.0,-16.08,53.0,X-LC,4.81,4.0,-0.12,2.24,8.45,X40N,ATH,FMCG
43,ITC,452.00,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.35,-37.66,113.18,32.89,54471.0,-29078.0,48128.0,-694.37,54.0,L-MC,6.16,259.0,-0.53,0.36,32.54,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.12,-19.55,125.81,81.67,94953.0,-18337.0,75473.0,6777.78,32.0,L-SC,14.88,271.0,-0.19,0.56,46.16,XR,NTT,CERAMICS
50,MASFIN,398.61,1.91,-2.94,25.74,22.05,24479.0,-2880.0,95100.0,-16.44,54.0,H-SC,7.04,164.0,-0.12,0.71,38.23,XR,ATH,FINANCE
13,BSOFT,831.70,0.53,-21.97,93.24,50.79,102333.0,-30901.0,109752.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.79,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.24,-14.69,17.25,0.02,25687.0,-25647.0,148912.0,127.12,46.0,M-SC,16.63,234.0,-1.00,1.11,29.53,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.53,-21.97,93.24,50.79,102333.0,-30901.0,109752.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.79,XR,ATH,IT
32,HCLTECH,1922.01,-1.84,7.57,15.95,24.73,41509.0,18319.0,260243.0,12.34,62.0,X-LC,5.74,7.0,0.44,1.94,23.96,X40,ATH,IT
84,WIPRO,420.00,-1.52,5.58,63.16,72.27,112993.0,9452.0,178900.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.33,12.10,XR,NTT,IT
56,RAJESHEXPO,518.00,4.03,-63.60,174.92,0.07,87339.0,-87246.0,49931.0,1729.55,53.0,L-SC,2.35,268.0,-1.00,0.37,24.18,OX40N,NTT,JEWELLERY
73,TCS,4389.97,-0.87,-12.83,36.83,19.28,111072.0,-44376.0,301580.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.49,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.81
1,25,45.74
2,50,76.69


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.34
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.27
X40      22.53
X40N     10.94
XR       10.04
XY25      9.40
AR        9.33
OX40N     7.73
SR        1.04
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.50
X-MC    23.59
X-LC    21.12
M-SC    12.09
X-SC     7.05
H-MC     4.98
M-MC     1.41
L-SC     1.38
M-LC     1.33
H-LC     1.17
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.46,-7.45,42.20
IT,12.86,-16.82,79.98
FINANCE,9.51,-21.63,73.03
ELECTRICAL,6.24,-12.05,53.09
MISC,6.10,-58.55,123.50
PAINTS,6.06,-13.52,30.39
INSURANCE,4.62,-2.04,38.00
PHARMA,4.10,-1.86,34.46
AUTO,3.40,-25.63,79.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243256.0,21
AR,1321077.0,10
XR,1314201.0,13
X40,1017077.0,14
X40N,981811.0,9
OX40N,752566.0,10
XY25,380975.0,6
SR,281155.0,2
MH,83537.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692315.0,25
M-SC,1435378.0,15
X-MC,1318859.0,16
X-SC,974297.0,8
X-LC,928831.0,11
H-MC,411371.0,3
L-SC,274933.0,3
M-LC,112993.0,1
H-LC,76433.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295891.0      6
           AR         900459.0      5
M-SC       XY24       814170.0      6
H-SC       XR         783992.0      7
X-SC       X40N       503460.0      3
X-MC       X40        459172.0      7
           XY24       397144.0      3
X-LC       X40        379948.0      5
H-SC       OX40N      347281.0      4
M-SC       OX40N      317946.0      5
X-SC       XY24       292880.0      3
H-SC       SR         281155.0      2
X-MC       X40N       276626.0      4
X-LC       XY24       247874.0      2
H-MC       AR         216074.0      2
X-LC       X40N       201725.0      2
H-MC       XY24       195297.0      1
L-SC       XR         187594.0      2
X-MC       XY25       185917.0      2
X-SC       X40        177957.0      2
M-SC       XR         175151.0      2
           AR         128111.0      2
M-LC       XR         112993.0      1
X-LC       XY25        99284.0      2
L-SC       OX40N       87339.0      1
H-SC       MH          83537.0      1
H-LC       AR          76433.0      1
M-MC       XY25        56740.0      1
L-MC       XR          54471.0      1
L-LC       XY25        39034.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
